In [1]:
pip install --force-reinstall agent_system/dist/mesa_restaurant_agents-0.0.1-py3-none-any.whl

Processing c:\users\s.neumann\documents\11_master_cas\code\mesa-restaurant-agents\agent_system\dist\mesa_restaurant_agents-0.0.1-py3-none-any.whl
  Using cached mesa-3.1.0-py3-none-any.whl.metadata (9.9 kB)
  Using cached pandas-2.2.3-cp312-cp312-win_amd64.whl.metadata (19 kB)
  Using cached numpy-2.2.2-cp312-cp312-win_amd64.whl.metadata (60 kB)
  Using cached scikit_learn-1.6.1-cp312-cp312-win_amd64.whl.metadata (15 kB)
  Using cached scipy-1.15.1-cp312-cp312-win_amd64.whl.metadata (60 kB)
  Using cached seaborn-0.13.2-py3-none-any.whl.metadata (5.4 kB)
  Using cached matplotlib-3.10.0-cp312-cp312-win_amd64.whl.metadata (11 kB)
  Using cached plotly-5.24.1-py3-none-any.whl.metadata (7.3 kB)
  Using cached contourpy-1.3.1-cp312-cp312-win_amd64.whl.metadata (5.4 kB)
  Using cached cycler-0.12.1-py3-none-any.whl.metadata (3.8 kB)
  Using cached fonttools-4.56.0-cp312-cp312-win_amd64.whl.metadata (103 kB)
  Using cached kiwisolver-1.4.8-cp312-cp312-win_amd64.whl.metadata (6.3 kB)
  Using 

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.19.0 requires numpy<2.2.0,>=1.26.0, but you have numpy 2.2.2 which is incompatible.


In [2]:
import mesa
from mesa_restaurant_agents.model.restaurant_model import RestaurantModel
import pandas as pd

params = {"n_waiters":10, "grid_width":20, "grid_height": 20}

results = mesa.batch_run(
    RestaurantModel,
    parameters=params,
    iterations=1,
    max_steps=300,
    number_processes=1,
    data_collection_period=1,
    display_progress=True,
)

# Handle data conversion with explicit types
df = pd.DataFrame(results)
numeric_cols = ['Step', 'Customer_Count', 'Average_Wait_Time', 'Average_Customer_Satisfaction', 'Profit']
df[numeric_cols] = df[numeric_cols].fillna(0).astype(float)
 
# Get only columns with data
non_empty_cols = [col for col in df.columns if df[col].notna().any()]
df[non_empty_cols].head()

  0%|          | 0/1 [00:00<?, ?it/s]

Day 1, Hour 12:00:
Customers paid: 0
Customers left without paying: 0
Current profit: $0.00

Day 1, Hour 13:00:
Customers paid: 3
Customers left without paying: 0
Current profit: $71.40

Day 1, Hour 14:00:
Customers paid: 29
Customers left without paying: 0
Current profit: $739.50

Starting shift 2: Afternoon
Shift 2: 2 waiters needed, 10 currently active
Removed 8 excess waiters
Day 1, Hour 15:00:
Customers paid: 77
Customers left without paying: 0
Current profit: $1922.70

Day 1, Hour 16:00:
Customers paid: 121
Customers left without paying: 0
Current profit: $3039.60

Day 1, Hour 17:00:
Customers paid: 144
Customers left without paying: 0
Current profit: $3621.00

Day 1, Hour 18:00:
Customers paid: 153
Customers left without paying: 0
Current profit: $3870.90

Starting shift 3: Evening
Shift 3: 2 waiters needed, 2 currently active
No change in waiters for shift 3
Day 1, Hour 19:00:
Customers paid: 184
Customers left without paying: 0
Current profit: $4707.30

Day 1, Hour 20:00:
Cust

c:\Users\s.neumann\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(


Day 1, Hour 23:00:
Customers paid: 271
Customers left without paying: 84
Current profit: $6869.70

===== Day 1 Revenue Report =====
Food revenue: $6707.70
Tips collected: $162.00
Total revenue: $6869.70
Customers paid: 271
Customers left without paying: 84
Total orders served: 271
Applying manager's schedule for day 2
Predicted customers: {1: 75, 2: 87, 3: 92}
Waiters per shift: {1: 4, 2: 5, 3: 5}
Creating waiters for first shift of day 2
Shift 1: 4 waiters needed, 2 currently active
Added 2 new waiters
Day 2, Hour 12:00:
Customers paid: 0
Customers left without paying: 0
Current profit: $0.00

Day 2, Hour 13:00:
Customers paid: 1
Customers left without paying: 0
Current profit: $25.50

Day 2, Hour 14:00:
Customers paid: 24
Customers left without paying: 0
Current profit: $596.70

Starting shift 2: Afternoon
Shift 2: 5 waiters needed, 4 currently active
Added 1 new waiters
Day 2, Hour 15:00:
Customers paid: 87
Customers left without paying: 0
Current profit: $2264.40

Day 2, Hour 16:00

c:\Users\s.neumann\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(


,RunId,iteration,Step,n_waiters,grid_width,grid_height,day,shift,time,Customer_Count,Waiters_Count,Average_Wait_Time,Average_Customer_Satisfaction,Profit,Customer_Info,Waiter_Info,GridState,Daily_Stats
0,0,0,0.0,10,20,20,1,1.0,660,0.0,10,0.0,100.0,0.0,[],"[{'waiter_nr': 1, 'tips': 0, 'served_customers...","[{'pos': (12, 12), 'type': 'Kitchen'}, {'pos':...",[{}]
1,0,0,1.0,10,20,20,1,1.0,660,0.0,10,0.0,100.0,0.0,[],"[{'waiter_nr': 1, 'tips': 0, 'served_customers...","[{'pos': (12, 12), 'type': 'Kitchen'}, {'pos':...",[{}]
2,0,0,2.0,10,20,20,1,1.0,665,0.0,10,0.0,100.0,0.0,[],"[{'waiter_nr': 1, 'tips': 0, 'served_customers...","[{'pos': (12, 12), 'type': 'Kitchen'}, {'pos':...",[{}]
3,0,0,3.0,10,20,20,1,1.0,665,0.0,10,0.0,100.0,0.0,[],"[{'waiter_nr': 1, 'tips': 0, 'served_customers...","[{'pos': (12, 12), 'type': 'Kitchen'}, {'pos':...",[{}]
4,0,0,4.0,10,20,20,1,1.0,670,1.0,10,0.0,100.0,0.0,"[{'customer_nr': 12, 'waiting_time': 0, 'order...","[{'waiter_nr': 1, 'tips': 0, 'served_customers...","[{'pos': (12, 12), 'type': 'Kitchen'}, {'pos':...",[{}]


In [3]:
# from mesa_restaurant_agents.model.restaurant_model import RestaurantModel

# model = RestaurantModel(n_waiters=10, grid_width=23, grid_height=23)
# for i in range(500):
#     model.step()

Day 1, Hour 12:00:
Customers paid: 0
Customers left without paying: 0
Current profit: $0.00

Day 1, Hour 13:00:
Customers paid: 3
Customers left without paying: 0
Current profit: $76.50

Day 1, Hour 14:00:
Customers paid: 32
Customers left without paying: 0
Current profit: $831.30

Starting shift 2: Afternoon
Shift 2: 2 waiters needed, 10 currently active
Removed 8 excess waiters
Day 1, Hour 15:00:
Customers paid: 81
Customers left without paying: 0
Current profit: $2075.70

Day 1, Hour 16:00:
Customers paid: 145
Customers left without paying: 0
Current profit: $3717.90

Day 1, Hour 17:00:
Customers paid: 170
Customers left without paying: 1
Current profit: $4335.00

Day 1, Hour 18:00:
Customers paid: 177
Customers left without paying: 1
Current profit: $4498.20

Starting shift 3: Evening
Shift 3: 2 waiters needed, 2 currently active
No change in waiters for shift 3
Day 1, Hour 19:00:
Customers paid: 202
Customers left without paying: 1
Current profit: $5115.30

Day 1, Hour 20:00:
Cust

c:\Users\s.neumann\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(


Day 2, Hour 16:00:
Customers paid: 116
Customers left without paying: 0
Current profit: $3065.10

Day 2, Hour 17:00:
Customers paid: 136
Customers left without paying: 0
Current profit: $3544.50

Day 2, Hour 18:00:
Customers paid: 142
Customers left without paying: 0
Current profit: $3692.40

Starting shift 3: Evening
Shift 3: 5 waiters needed, 5 currently active
No change in waiters for shift 3
Day 2, Hour 19:00:
Customers paid: 162
Customers left without paying: 0
Current profit: $4222.80

Day 2, Hour 20:00:
Customers paid: 202
Customers left without paying: 0
Current profit: $5191.80

Day 2, Hour 21:00:
Customers paid: 253
Customers left without paying: 0
Current profit: $6502.50

Day 2, Hour 22:00:
Customers paid: 304
Customers left without paying: 0
Current profit: $7813.20

Day 2, Hour 23:00:
Customers paid: 324
Customers left without paying: 0
Current profit: $8333.40

===== Day 2 Revenue Report =====
Food revenue: $7870.80
Tips collected: $462.60
Total revenue: $8333.40
Custome

c:\Users\s.neumann\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(


Day 3, Hour 17:00:
Customers paid: 159
Customers left without paying: 0
Current profit: $4059.60

Day 3, Hour 18:00:
Customers paid: 165
Customers left without paying: 0
Current profit: $4212.60

Starting shift 3: Evening
Shift 3: 5 waiters needed, 6 currently active
Removed 1 excess waiters
Day 3, Hour 19:00:
Customers paid: 196
Customers left without paying: 0
Current profit: $5033.70

Day 3, Hour 20:00:
Customers paid: 230
Customers left without paying: 0
Current profit: $5895.60

Day 3, Hour 21:00:
Customers paid: 286
Customers left without paying: 0
Current profit: $7308.30

Day 3, Hour 22:00:
Customers paid: 343
Customers left without paying: 0
Current profit: $8726.10

Day 3, Hour 23:00:
Customers paid: 383
Customers left without paying: 0
Current profit: $9710.40

===== Day 3 Revenue Report =====
Food revenue: $8887.20
Tips collected: $823.20
Total revenue: $9710.40
Customers paid: 383
Customers left without paying: 0
Total orders served: 391
Applying manager's schedule for day

c:\Users\s.neumann\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(


In [4]:
print(results[0])

{'RunId': 0, 'iteration': 0, 'Step': 0, 'n_waiters': 10, 'grid_width': 20, 'grid_height': 20, 'day': 1, 'shift': 1, 'time': 660, 'Customer_Count': 0, 'Waiters_Count': 10, 'Average_Wait_Time': 0.0, 'Average_Customer_Satisfaction': 100.0, 'Profit': 0, 'Customer_Info': [], 'Waiter_Info': [{'waiter_nr': 1, 'tips': 0, 'served_customers': 0}, {'waiter_nr': 2, 'tips': 0, 'served_customers': 0}, {'waiter_nr': 3, 'tips': 0, 'served_customers': 0}, {'waiter_nr': 4, 'tips': 0, 'served_customers': 0}, {'waiter_nr': 5, 'tips': 0, 'served_customers': 0}, {'waiter_nr': 6, 'tips': 0, 'served_customers': 0}, {'waiter_nr': 7, 'tips': 0, 'served_customers': 0}, {'waiter_nr': 8, 'tips': 0, 'served_customers': 0}, {'waiter_nr': 9, 'tips': 0, 'served_customers': 0}, {'waiter_nr': 10, 'tips': 0, 'served_customers': 0}], 'GridState': [{'pos': (12, 12), 'type': 'Kitchen'}, {'pos': (7, 17), 'type': 'Table'}, {'pos': (5, 1), 'type': 'Table'}, {'pos': (17, 3), 'type': 'Table'}, {'pos': (3, 13), 'type': 'Table'}, 

In [5]:
from mesa_restaurant_agents.visualization import *
from mesa_restaurant_agents.visualization import GridAnimator

# Create an instance of GridAnimator
animator = GridAnimator(results)

ani = animator.animate_first_run()
ani.save('animated_heatmap.gif', writer='pillow', fps=2, dpi=80)

In [6]:
df_mean_step = display_mean_step_results(results)

In [7]:
customer_infos_df = display_first_run_step_results_customer(results)
waiter_infos_df = display_first_run_step_results_waiter(results)

None


None


None


None
